In [2]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker


In [3]:
engine = create_engine('sqlite:///nfl_fantasy.db')

In [4]:
#Load page and establish beautiful soup object
html= urlopen('http://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2012&wk=1&rules=1')
bsobj= BeautifulSoup(html, 'lxml')

#create qb dataframe with column names
columns = ['player_id', 'name', 'team', 'vs', 'ffpts', 'patt', 'pcmp', 'pyds', 'ptds', 'pint', 'p2', 'ruatt', 'ruyds','rutds', 'ru2', 'rec', 'recyds', 'rectd', 'rec2', 'fum', 'fmbtd']
wr = pd.DataFrame(columns = columns, dtype= float)
wr['name'] = wr['name'].astype(str)
wr['team'] = wr['team'].astype(str)
wr['vs'] = wr['vs'].astype(str)

In [5]:
#Parse html to obtain list of QB's competeing in relevant week 
wr_list= []
for item in (bsobj.tbody({'a':'href'})):
    if '.' not in item.get_text():
        wr_list.append(item.get_text())
    else:
        if item.get_text() == 'A.J. Green':
            wr_list.append(item.get_text())

wr['name'] = wr_list
wr['player_id'] = wr.index + 1

In [6]:
#Parse beautiful soup object to get only data needed in list format
parsed_stats = []
for item in bsobj.tbody.find_all({'td':'class'}):
    parsed_stats.append(item.get_text().split())

In [7]:
#Populate team and vs columns in qb dataframe by sorting parsed_list data
stats = []
team_list = []
vs_list = []
for idx in range(len(parsed_stats)):
    try:
        stats.append(float(parsed_stats[idx][0]))
    except:
        if len(parsed_stats[idx]) > 1:
            team_list.append(parsed_stats[idx][4])
            vs_list.append(parsed_stats[idx+1][0].replace('@', ''))

wr.team, wr.vs = team_list, vs_list

#populate the rest of data frame with state from relevant week
remaining_columns = columns[4:]
for num, col in enumerate(remaining_columns, start=2):
    print(num, col)
    # Flatten the list and assign it to the relevant column
    flat_list = [item for sublist in parsed_stats[num:][::19] for item in sublist]
    wr[col] = flat_list

2 ffpts
3 patt
4 pcmp
5 pyds
6 ptds
7 pint
8 p2
9 ruatt
10 ruyds
11 rutds
12 ru2
13 rec
14 recyds
15 rectd
16 rec2
17 fum
18 fmbtd


In [8]:
wr

,player_id,name,team,vs,ffpts,patt,pcmp,pyds,ptds,pint,...,ruatt,ruyds,rutds,ru2,rec,recyds,rectd,rec2,fum,fmbtd
0,1,Kevin Ogletree,Dal,NYG,23.00,0,0,0,0,0,...,0,0,0,0,8,114,2,0,0,0
1,2,Julio Jones,Atl,KC,22.00,0,0,0,0,0,...,0,0,0,0,6,108,2,0,0,0
2,3,Stephen Hill,NYJ,Buf,20.00,0,0,0,0,0,...,0,0,0,0,5,89,2,0,0,0
3,4,Lance Moore,NO,Was,18.00,0,0,0,0,0,...,0,0,0,0,6,120,1,0,0,0
4,5,Demaryius Thomas,Den,Pit,17.00,0,0,0,0,0,...,0,0,0,0,5,110,1,0,0,0
5,6,Brandon Marshall,Chi,Ind,17.00,0,0,0,0,0,...,0,0,0,0,9,119,1,0,0,0
6,7,Andre Johnson,Hou,Mia,17.00,0,0,0,0,0,...,0,0,0,0,8,119,1,0,0,0
7,8,Pierre Garcon,Was,NO,16.00,0,0,0,0,0,...,0,0,0,0,4,109,1,0,0,0
8,9,Jeremy Maclin,Phi,Cle,15.00,0,0,0,0,0,...,0,0,0,0,7,96,1,0,0,0
9,10,James Jones,GB,SF,14.00,0,0,0,0,0,...,0,0,0,0,4,81,1,0,0,0


In [9]:
wr.to_sql('wr.db', con=engine)